In [2]:
import numpy as np
import tensorflow as tf
import scipy
from tensorflow import keras
from tensorflow.keras import layers

from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

from tensorflow.keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input

import matplotlib.pyplot as plt
from scipy import stats

from tqdm.notebook import tqdm

## Extracting features from VGG16

In [3]:
img_size = 224
resnet = ResNet50(weights='imagenet', include_top=True, pooling='max', input_shape = (img_size, img_size, 3))
resnet.summary()

102973440/102967424 [==============================] - 2s 0us/step
Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 112, 112, 64) 256         conv1_conv[0][0]                 
________________________

In [4]:
op = resnet.get_layer('avg_pool').output
ip = resnet.input

basemodel = keras.Model(
    inputs=ip,
    outputs=op,
)

basemodel.trainable=False

In [5]:
images = np.empty((50, img_size, img_size, 3))
for i in range(50):
  image = load_img("data/" + str(i+1) +".jpg", target_size=(img_size, img_size))
  image = img_to_array(image)

  for j in range(img_size):
    for k in range(img_size):
      g = (image[j,k,0] + image[j,k,1] + image[j,k,2])/3
      image[j,k,0] = g
      image[j,k,1] = g
      image[j,k,2] = g

  images[i,:,:,:] = image

images = preprocess_input(images)
features = basemodel.predict(images)

colors = np.empty((5, img_size, img_size, 3))
for i in range(5):
  image = load_img("data/C" + str(i) + ".png", target_size=(img_size, img_size))
  image = img_to_array(image)
  colors[i,:,:,:] = image

colors = preprocess_input(colors)
colorfeatures = basemodel.predict(colors)

features = features.reshape(50,-1)
colorfeatures = colorfeatures.reshape(5,-1)

data = np.genfromtxt('data/bandw.csv', skip_header = 6, delimiter=',')
humanpred = np.zeros((50,5))
for i in range(50):
  for j in range(56):
    color = int(data[j, i]);
    humanpred[i, color] = humanpred[i, color]+1

prob = humanpred/56

In [6]:
inputimages = np.zeros((250,features.shape[1]))
inputcolors = np.zeros((250,features.shape[1]))
outputprob = np.zeros((250))

for i in range(50):
  for j in range(5):
    inputimages[5*i+j,:] = features[i,:]

for i in range(5):
  for j in range(50):
    inputcolors[5*j+i] = colorfeatures[i, :]

outputprob = prob.reshape(250)

## Model

In [7]:
def get_model(op_features=75):
  imageinputs = keras.Input(shape=(features.shape[1]), name="imageinputs")
  colorinputs = keras.Input(shape=(features.shape[1]), name="colorinputs")
  dense = layers.Dense(op_features, name = "linearlayer", activation='relu')
  imageresults = dense(imageinputs)
  colorresults = dense(colorinputs)
  result = tf.keras.layers.Dot(axes=1, normalize=True, name="dot")([imageresults, colorresults])

  model = keras.Model(
      inputs=[imageinputs, colorinputs],
      outputs=[result],
  )
  model.compile(loss = 'mean_squared_error', optimizer = tf.keras.optimizers.Adam(lr=0.001), metrics = ['mean_squared_error'])
  
  return model

def get_corr(output, outputprob, vs, ve):
  x = np.copy(output.reshape(50,5))

  for i in range(50):
    rowsum = np.sum(x[i,:])
    for j in range(5):
      x[i,j] = x[i,j]/rowsum

  y = np.copy(outputprob.reshape(50,5))
  testx = x.reshape(-1)[vs:ve]
  testy = y.reshape(-1)[vs:ve]

  return scipy.stats.pearsonr(testx, testy)[0], scipy.stats.pearsonr(testx, testy)[1], testx

In [8]:
Rp = []
pvalue = []

for i in tqdm(range(50)):
  corr = [] #This contains correlation corresponding to specific validation data
  pred = np.zeros(250) #This will contain the final prediction for each of the loop. Total 5 subset of validation 50 each.

  # 5-fold cross validation 
  for vs in [0, 50, 100, 150, 200]:
    ve = vs + 50
    model = get_model(op_features=75)
    history = model.fit(
        {"imageinputs": inputimages[[*range(vs)] + [*range(ve, 250)]], "colorinputs": inputcolors[[*range(vs)] + [*range(ve, 250)]]},
        {"dot": outputprob[[*range(vs)] + [*range(ve, 250)]]},
        epochs=30,
        batch_size=10,
        shuffle=True,
        verbose=0,
    )
    output = model.predict({"imageinputs": inputimages, "colorinputs": inputcolors})
    R, _, pred[vs:ve] = get_corr(output, outputprob, vs, ve)
    corr.append(R)

  temp = get_corr(pred, outputprob, 0, 250)
  Rp.append(temp[0])
  pvalue.append(temp[1])

In [9]:
print(np.mean(Rp))
print(np.std(Rp))

0.5991957485155681
0.018110046634651048


In [10]:
Rp

[0.5801709938582272,
 0.5899025545335739,
 0.5969613216330956,
 0.6101431132358932,
 0.5979027714913211,
 0.6125562048319934,
 0.596887336960209,
 0.5946011163254815,
 0.5858966071696762,
 0.6144839352016396,
 0.6333678422391267,
 0.5865431406087254,
 0.6065985541355771,
 0.5683726759891272,
 0.5478744789265166,
 0.6349701665616003,
 0.62810169749015,
 0.6035788333502904,
 0.5978880922695051,
 0.6051692694768986,
 0.6090294237854242,
 0.554184551148379,
 0.6178518469060518,
 0.588238479699937,
 0.6215438187394173,
 0.6169630204184717,
 0.5711606057453417,
 0.597705285651809,
 0.5853913422075855,
 0.6237312128096242,
 0.6201611221674795,
 0.5956234658309013,
 0.5921437464525007,
 0.5930813850417916,
 0.5944044870442963,
 0.6101064656987017,
 0.6212938095285931,
 0.5904236875078752,
 0.591171808577931,
 0.5937908365253856,
 0.5925711972316108,
 0.601453181883406,
 0.584052502855614,
 0.580883978689812,
 0.6170880698558098,
 0.5818929944085772,
 0.5981894285751151,
 0.6095176744354258,
 0

In [11]:
print(np.mean(pvalue))
print(np.std(pvalue))

1.4507635931353528e-22
7.97334377862783e-22
